In [5]:
import pickle
import os
import numpy as np
import pandas as pd
import scipy.sparse as sp

In [101]:
with open('test.pkl', 'rb') as f:
    csr = pickle.load(f)
    
with open('neg.pkl', 'rb') as f1:
    neg = pickle.load(f1)
# neg = np.array(neg)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14232\3729012348.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  csr = pickle.load(f)


In [102]:
for i in range(8039):
    j = sum(csr[i].data)
    if j > 1:
        print(i, j)

In [106]:
csr = ub_valid

# csr = csr.tocsr()
idx, _ = np.nonzero(np.sum(csr, 1))
pos = np.nonzero(csr[idx].toarray())[1]
pos = pos[:, np.newaxis]
neg = neg[idx]

In [107]:
arr = np.concatenate((pos, neg), axis=1)

In [6]:
def get_ub(task):
        with open(os.path.join('data/Youshu', 'user_bundle_{}.txt'.format(task)), 'r') as f:
            u_b_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split('\t')), f.readlines()))

        indice = np.array(u_b_pairs, dtype=np.int32)
        values = np.ones(len(u_b_pairs), dtype=np.float32)
        u_b_graph = sp.coo_matrix(
            (values, (indice[:, 0], indice[:, 1])), shape=(8039, 4771)).tocsr()

        return u_b_pairs, u_b_graph
    
def get_bi():
    with open(os.path.join('data/Youshu', 'bundle_item.txt'), 'r') as f:
        b_i_pairs = list(map(lambda s: tuple(int(i) for i in s[:-1].split('\t')), f.readlines()))

    indice = np.array(b_i_pairs, dtype=np.int32)
    values = np.ones(len(b_i_pairs), dtype=np.float32)
    b_i_graph = sp.coo_matrix(
        (values, (indice[:, 0], indice[:, 1])), shape=(4771, 32770)).tocsr()

    return b_i_graph

In [7]:
bi_graph = get_bi()

In [8]:
ub_pairs_train, ub_graph_train = get_ub('train')
ub_pairs_valid, ub_graph_valid = get_ub('tune')
ub_pairs_test, ub_graph_test = get_ub('test')
ub = ub_graph_train + ub_graph_valid + ub_graph_test

In [9]:
def regen(ub):
    train_rows, train_cols = ub.nonzero()[0][:-2], ub.nonzero()[1][:-2]
    valid_rows, valid_cols = ub.nonzero()[0][-2:-1], ub.nonzero()[1][-2:-1]
    test_rows, test_cols = ub.nonzero()[0][-1:], ub.nonzero()[1][-1:]
    ub_train = sp.csr_matrix((np.ones(len(train_rows)), (train_rows, train_cols)), shape=ub.shape)
    ub_valid = sp.csr_matrix((np.ones(len(valid_rows)), (valid_rows, valid_cols)), shape=ub.shape)
    ub_test = sp.csr_matrix((np.ones(len(test_rows)), (test_rows, test_cols)), shape=ub.shape)
    
    return ub_train, ub_valid, ub_test
    

In [10]:
ub_train, ub_valid, ub_test = regen(ub)

In [15]:
row, col = ub_train.nonzero()

In [20]:
row = np.array(row)
col = np.array(col)
pairs = np.concatenate((row[:, np.newaxis], col[:, np.newaxis]), axis=1)

In [24]:
tuple_pairs = [tuple(pair) for pair in pairs]

# Convert the list of tuples to a numpy array (optional)
tuple_pairs_array = np.array(tuple_pairs, dtype=[('x', int), ('y', int)])

In [105]:
neg = []
idx = ub_graph_valid.sum(axis=1).nonzero()[0]

for u in range(8039):
    neg.append(np.random.choice(np.setdiff1d(np.arange(4771), ub_train[u].nonzero()[1]), 99, replace=False))
    
neg = np.array(neg)